![Py4Eng](img/logo.png)

# Keras Functional API
## Yoav Ram

The [functional API](https://keras.io/guides/functional_api/) is an advanced Keras feature that is used for building non-sequential networks, such as ResNet, or networks made from mutiple nested models, such as GAN and autoencoders.

In the functional API, each layer is a function that operates on an input to produce an output. These inputs and outputs are *symbolic* and do not contain any data when we set the network up. They will only contain data during training or predicting.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
import keras

print('Tensorflow:', tf.__version__)    
print('Keras:', keras.__version__)
print('GPU:', tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

Tensorflow: 2.13.1
Keras: 2.13.1
GPU: []



In [3]:
def display_image(im):
    fig, ax = plt.subplots()
    ax.imshow(im, cmap='gray_r')
    ax.set_xticks([])
    ax.set_yticks([])    

In [4]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
nsamples, width, height = X_train.shape
X_train = (X_train/255).astype('float32')
X_test = (X_test/255).astype('float32')

In [5]:
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)
ncats = Y_test.shape[1]

# FFN with `Sequential`

This is the two-layers simple feed forward network we implemented already.

In [6]:
model = keras.models.Sequential()

model.add(
    keras.layers.Flatten(input_shape=(width, height)))
model.add(
    keras.layers.Dense(100, activation='relu'))
model.add(
    keras.layers.Dropout(rate=0.5))
model.add(
    keras.layers.Dense(ncats, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [6]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Epoch 1/5
1200/1200 [==============================] - 4s 3ms/step - loss: 0.4593 - accuracy: 0.8638 - val_loss: 0.1912 - val_accuracy: 0.9442
Epoch 2/5
1200/1200 [==============================] - 3s 3ms/step - loss: 0.2606 - accuracy: 0.9225 - val_loss: 0.1460 - val_accuracy: 0.9562
Epoch 3/5
1200/1200 [==============================] - 2s 2ms/step - loss: 0.2193 - accuracy: 0.9359 - val_loss: 0.1240 - val_accuracy: 0.9627
Epoch 4/5
1200/1200 [==============================] - 2s 1ms/step - loss: 0.1959 - accuracy: 0.9415 - val_loss: 0.1084 - val_accuracy: 0.9691
Epoch 5/5
1200/1200 [==============================] - 2s 1ms/step - loss: 0.1775 - accuracy: 0.9465 - val_loss: 0.1062 - val_accuracy: 0.9687


# FFN with functional API 

We now implement it with the functional API.

In this API, each layer is a function that works on some input tensor. A tensor is basically a placeholder for future data (array).

We first create the layers, then apply them to the tensors. The first tensor is a special `Input` tensor. The model is then created using the input and the output tensors.

In [9]:
flatten_layer = keras.layers.Flatten()
hidden_layer = keras.layers.Dense(100, activation="relu")
dropout_layer = keras.layers.Dropout(rate=0.5)
softmax_layer = keras.layers.Dense(ncats, activation="softmax")

x = keras.Input(shape=(width, height))
x_flat = flatten_layer(x)
h = hidden_layer(x_flat)
h_drop = dropout_layer(h)
yhat = softmax_layer(h_drop)

model = keras.Model(inputs=x, outputs=yhat)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The rest is per usual - compile and fit.

In [37]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [38]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 48us/step - loss: 0.4601 - accuracy: 0.8637 - val_loss: 0.1944 - val_accuracy: 0.9420
Epoch 2/5
60000/60000 [==============================] - 3s 44us/step - loss: 0.2600 - accuracy: 0.9232 - val_loss: 0.1461 - val_accuracy: 0.9555
Epoch 3/5
60000/60000 [==============================] - 3s 44us/step - loss: 0.2188 - accuracy: 0.9356 - val_loss: 0.1242 - val_accuracy: 0.9625
Epoch 4/5
60000/60000 [==============================] - 3s 44us/step - loss: 0.1957 - accuracy: 0.9407 - val_loss: 0.1152 - val_accuracy: 0.9642
Epoch 5/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.1782 - accuracy: 0.9467 - val_loss: 0.1063 - val_accuracy: 0.9696


# Exercise

Implement a CNN using the functional API. See the sequenctial example below (copied from [the CNN session](K_CNN.ipynb)).

[solution](../solutions/functional_keras.py)

In [26]:
# Sequential example
model = keras.models.Sequential()

model.add(
    keras.layers.Reshape(target_shape=(width, height, 1), 
                         input_shape=(width, height)))
model.add(
    keras.layers.Conv2D(32, (5, 5), activation='relu'))
model.add(
    keras.layers.MaxPool2D())
model.add(
    keras.layers.Conv2D(64, (5, 5), activation='relu'))
model.add(
    keras.layers.MaxPool2D())
model.add(
    keras.layers.Flatten())
model.add(
    keras.layers.Dense(1024, activation='relu'))
model.add(
    keras.layers.Dropout(rate=0.5))
model.add(
    keras.layers.Dense(ncats, activation='softmax'))

In [28]:
# Functional 
# your code here

################

In [44]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [45]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2013s 34ms/step - loss: 0.1307 - accuracy: 0.9591 - val_loss: 0.0342 - val_accuracy: 0.9890
Epoch 2/5
60000/60000 [==============================] - 65s 1ms/step - loss: 0.0441 - accuracy: 0.9862 - val_loss: 0.0298 - val_accuracy: 0.9903
Epoch 3/5
60000/60000 [==============================] - 65s 1ms/step - loss: 0.0312 - accuracy: 0.9908 - val_loss: 0.0247 - val_accuracy: 0.9917
Epoch 4/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0251 - accuracy: 0.9920 - val_loss: 0.0210 - val_accuracy: 0.9942
Epoch 5/5
60000/60000 [==============================] - 59s 979us/step - loss: 0.0212 - accuracy: 0.9935 - val_loss: 0.0301 - val_accuracy: 0.9906


# References

- [Keras functional API](https://keras.io/guides/functional_api/)

# Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)